In [11]:
import random
import json
import pickle
import numpy as np
!pip install tensorflow
import tensorflow as tf
from google.colab import files

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('DB.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))
pickle.dump(lemmatizer, open('lemmatizer.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word) for word in wordPatterns]
    for word in words:
        bag.append(1 if word in wordPatterns else 0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(trainX, trainY, epochs=350, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print('Done')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Epoch 1/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0181 - loss: 4.0378
Epoch 2/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0521 - loss: 3.9344
Epoch 3/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0629 - loss: 3.8809
Epoch 4/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1027 - loss: 3.7393
Epoch 5/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0944 - loss: 3.6361
Epoch 6/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0779 - loss: 3.5319
Epoch 7/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1208 - loss: 3.5169
Epoch 8/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2108 - loss: 3.1847
Epoch 9/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1849 - loss: 3.2126
Epoch 10/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2841 - loss: 2.9754
Epoch 11/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2425 - loss: 2.8781
Epoch 12/350
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy:

Done


In [ ]:
import random
import json
import pickle
import os
import nltk
import numpy as np

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

from google.colab import files

model = load_model('chatbot_model.h5')
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('DB.json').read())

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.10
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    if not intents_list or 'intent' not in intents_list[0] or 'probability' not in intents_list[0]:
        return "I'm sorry, I didn't understand that. Could you rephrase?"

    probability = float(intents_list[0]['probability'])
    if probability < 0.6:
        return "I'm not quite sure about that. Could you provide more details?"

    tag = intents_list[0]['intent']
    list_of_intents = intents_json.get('intents', [])

    for intent in list_of_intents:
        if intent.get('tag') == tag:
            responses = intent.get('responses', [])
            if not responses:
                return "I don't have information about that yet."

            response = random.choice(responses)

            # Handle both string and dictionary responses
            if isinstance(response, dict):
                return format_dict_response(response)
            return response

    return "I'm not familiar with that topic. Would you like information about our Toyota models instead?"

def format_dict_response(response_dict):
    """Convert a dictionary response into a readable string"""
    formatted_lines = []
    for key, value in response_dict.items():
        if isinstance(value, list):
            formatted_lines.append(f"{key.capitalize()}: {', '.join(value)}")
        else:
            formatted_lines.append(f"{key.capitalize()}: {value}")
    return "\n".join(formatted_lines)

# Your chatbot loop
#print("GO! Bot is running!")

print("Hi, Welcome to EduBot!")

name = input("What is your name: ")
print("Welcome,", name)

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print("EduBot: " + res)


Hi, Welcome to EduBot!
What is your name: amaan
Welcome, amaan
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
EduBot: Welcome to Toyota UAE! Are you looking for a sedan, SUV, truck, or hybrid?
i am looking for a sedan
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
EduBot: For **family cars**: Try the <b>Toyota Highlander</b> (7-seater) or <b>Sienna</b> (minivan). Both have great safety features.
toyota highlander
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
EduBot: I'm sorry, I didn't understand that. Could you rephrase?
what is the best off roader
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
EduBot: The Camry is a spacious sedan, while the Corolla is compact. For SUVs, the RAV4 is urban-friendly, and the Fortuner is more rugged. What’s your priority: comfort or off-road use?
what is the best dune basher
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
EduBot: The Camry is a spacious sedan, while the Corolla is compact. For SUVs, the RAV4 is urban-friendly, and the Fortuner is more rugged. What’s your priority: comfort or